# 练习11-循环神经网络介绍
--------
## 介绍

在本练习中，我们将使用NumPy实现一个循环神经网络。
递归神经网络（RNN）对于自然语言处理和其他序列任务非常有效，因为它们具有“记忆”功能。 它们可以一次读取一个输入$x^{⟨t⟩}$（如单词），并且通过隐藏层激活从一个时间步传递到下一个时间步来记住一些信息/上下文，这允许单向RNN从过去获取信息来处理后面的输入，双向RNN可以从过去和未来中获取上下文。

有些东西需要声明：

- 上标$[l]$表示第$l$层
    - 举例：$a^{[4]}$表示第4层的激活值，$W^{[5]}$与$b^{[5]}$是第5层的参数。
- 上标$(i)$表示第$i$个样本
    - 举例：$x^{(i)}$表示第$i$个输入的样本。
- 上标$<t>$表示第$t$个时间步
    - 举例：$x^{<t>}$表示输入$x$的第$t$个时间步，$x^{(i)<t>}$表示输入$x$的第$i$个样本的第$t$个时间步。
- 下标$i$表示向量的第$i$项
    - 举例：$a^{[l]}_{i}$表示$l$层中的第$i$个项的激活值。

在开始练习前，需要**介绍如下的文件**：

- rnn_utils.py  -提供了在这个练习中使用的各种功能函数，例如初始化等


在整个练习中，涉及如下的**必做作业**：

| 作业 | 分值 |
|--|--|
|[实现rnn单元](#1)| 25分|
|[实现rnn的前向传播](#2)|25分|
|[实现LSTM单元](#3)|25分|
|[实现LSTM的前向传播](#4)|25分|

开始之前，先导入所需要的Python库文件。

In [2]:
import numpy as np
import rnn_utils

## 1 循环神经网络的前向传播
我们来看一下下面的循环神经网络的图，在这里使用的是$T_x = T_y$。
![](11-1.png)
实现有以下步骤：
1. 实现RNN的一个时间步所需要计算的东西。
2. 在$T_x$时间步上实现一个循环，以便一次处理所有输入。

### 1.1 RNN单元
<span id='1'></span>
循环神经网络可以看作是单元的重复，首先要实现单个时间步的计算，下图描述了RNN单元的单个时间步的操作。

![](11-2.png)
现在我们要根据上图来实现一个RNN单元，这需要由以下几步完成：

1. 使用tanh函数计算隐藏单元的激活值：$a^{\langle t \rangle} = \tanh(W_{aa}a^{\langle t - 1 \rangle} + W_{ax}x^{\langle t \rangle} + b_a)$
2. 使用$a^{\langle t \rangle}$计算$\hat y ^{\langle t \rangle} = softmax( W_{ya}$，$softmax$在`rnn_utils.py`内。
3. 把$( a^{\langle t \rangle},a^{\langle t \rangle - 1},x^{\langle t \rangle},parameters)$存储到`cache`中。
4. 返回$a^{\langle t \rangle},y^{\langle t \rangle}$与`cache`。

 我们将向量化$m$个样本，因此，$x^{\langle t \rangle}$的维度为$(n_x,m)$，$a^{\langle t \rangle}$的维度为$(n_a,m)$。
 
接下来，你需要**编写代码补全函数rnn_cell_forward()**。

**要点**：
- 依据上述步骤进行实现
- 函数参数及返回值如函数说明所示
- 时刻检查各个变量的形状

In [3]:
def rnn_cell_forward(xt, a_prev, parameters):
    """
    根据图2实现RNN单元的单步前向传播
    
    参数：
        xt -- 时间步“t”输入的数据，维度为（n_x, m）
        a_prev -- 时间步“t - 1”的隐藏隐藏状态，维度为（n_a, m）
        parameters -- 字典，包含了以下内容:
                        Wax -- 矩阵，输入乘以权重，维度为（n_a, n_x）
                        Waa -- 矩阵，隐藏状态乘以权重，维度为（n_a, n_a）
                        Wya -- 矩阵，隐藏状态与输出相关的权重矩阵，维度为（n_y, n_a）
                        ba  -- 偏置，维度为（n_a, 1）
                        by  -- 偏置，隐藏状态与输出相关的偏置，维度为（n_y, 1）
    
    返回：
        a_next -- 下一个隐藏状态，维度为（n_a， m）
        yt_pred -- 在时间步“t”的预测，维度为（n_y， m）
        cache -- 反向传播需要的元组，包含了(a_next, a_prev, xt, parameters)
    """
    
    # 从“parameters”获取参数
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    ba = parameters["ba"]
    by = parameters["by"]
    
    ###在这里填入代码###
    
    # 使用上面的公式计算下一个激活值
    a_next = np.tanh(np.dot(Wax, xt) + np.dot(Waa, a_prev) + ba)
    
    # 使用上面的公式计算当前单元的输出
    yt_pred = np.dot(Wya, a_next) + by
    yt_pred = np.exp(yt_pred) / np.sum(np.exp(yt_pred), axis=0, keepdims=True)  # 计算softmax

    # 保存反向传播需要的值
    cache = (a_next, a_prev, xt, parameters)
    
    ###在这里填入代码###
    
    return a_next, yt_pred, cache

In [4]:
#测试rnn_cell_forward
print("==============测试rnn_cell_forward==============")
np.random.seed(1)
xt = np.random.randn(3,10)
a_prev = np.random.randn(5,10)
Waa = np.random.randn(5,5)
Wax = np.random.randn(5,3)
Wya = np.random.randn(2,5)
ba = np.random.randn(5,1)
by = np.random.randn(2,1)
parameters = {"Waa": Waa, "Wax": Wax, "Wya": Wya, "ba": ba, "by": by}

a_next, yt_pred, cache = rnn_cell_forward(xt, a_prev, parameters)
print("a_next[4] = ", a_next[4])
print("a_next.shape = ", a_next.shape)
print("yt_pred[1] =", yt_pred[1])
print("yt_pred.shape = ", yt_pred.shape)


==============测试rnn_cell_forward==============
a_next[4] =  [ 0.59584544  0.18141802  0.61311866  0.99808218  0.85016201  0.99980978
 -0.18887155  0.99815551  0.6531151   0.82872037]
a_next.shape =  (5, 10)
yt_pred[1] = [0.9888161  0.01682021 0.21140899 0.36817467 0.98988387 0.88945212
 0.36920224 0.9966312  0.9982559  0.17746526]
yt_pred.shape =  (2, 10)


### 1.2 RNN前向传播
<span id='2'></span>
可以看到的是RNN是刚刚构建的单元格的重复连接，如果输入的数据序列经过10个时间步，那么将复制RNN单元10次，每个单元将前一个单元中的隐藏状态$a^{\langle t-1 \rangle}$和当前时间步的输入数据$x^{\langle t \rangle}$作为输入。 

它为此时间步输出隐藏状态$a^{\langle t \rangle}$和预测$y^{\langle t \rangle}$。
![](11-3.png)

我们要根据上图来实现前向传播的代码，它由以下几步构成：

1. 创建$0$向量$zeros(a)$，它将保存RNN计算的所有的隐藏状态。
2. 使用“$a_0$”初始化`“next”`隐藏状态。
3. 循环所有时间步：
    1. 使用`rnn_cell_forward`函数来更新`“next”`隐藏状态与`cache`。
    2. 使用$a$来保存`“next”`隐藏状态（第$t$）个位置。
    3. 使用$y$来保存预测值。
    4. 把`cache`保存到`“caches”`列表中。
4. 返回$a, y$与`caches`。

接下来，你需要**编写代码对函数rnn_forward()进行补全**。

**要点**：
- 依据上述步骤进行实现
- 函数参数及返回值如函数说明所示
- 时刻检查各个变量的形状


In [5]:
def rnn_forward(x, a0, parameters):
    """
    根据图3来实现循环神经网络的前向传播
    
    参数：
        x -- 输入的全部数据，维度为(n_x, m, T_x)
        a0 -- 初始化隐藏状态，维度为 (n_a, m)
        parameters -- 字典，包含了以下内容:
                        Wax -- 矩阵，输入乘以权重，维度为（n_a, n_x）
                        Waa -- 矩阵，隐藏状态乘以权重，维度为（n_a, n_a）
                        Wya -- 矩阵，隐藏状态与输出相关的权重矩阵，维度为（n_y, n_a）
                        ba  -- 偏置，维度为（n_a, 1）
                        by  -- 偏置，隐藏状态与输出相关的偏置，维度为（n_y, 1）
    
    返回：
        a -- 所有时间步的隐藏状态，维度为(n_a, m, T_x)
        y_pred -- 所有时间步的预测，维度为(n_y, m, T_x)
        caches -- 为反向传播的保存的元组，维度为（【列表类型】cache, x)）
    """
    
    # 初始化“caches”，它将以列表类型包含所有的cache
    caches = []
    
    # 获取 x 与 Wya 的维度信息
    n_x, m, T_x = x.shape
    n_y, n_a = parameters["Wya"].shape
    
    # 使用0来初始化“a” 与“y”
    a = np.zeros((n_a, m, T_x))
    y_pred = np.zeros((n_y, m, T_x))
    
    # 初始化“next”
    a_next = a0
    
    # 遍历所有时间步
    for t in range(T_x):
        ## 1.使用rnn_cell_forward函数来更新“next”隐藏状态与cache。
        a_next, yt_pred, cache = rnn_cell_forward(x[:, :, t], a_next, parameters)
        
        ## 2.使用 a 来保存“next”隐藏状态（第 t ）个位置。
        a[:, :, t] = a_next
        
        ## 3.使用 y 来保存预测值。
        y_pred[:, :, t] = yt_pred
        
        ## 4.把cache保存到“caches”列表中。
        caches.append(cache)
    
    # 保存反向传播所需要的参数
    caches = (caches, x)
    
    return a, y_pred, caches

In [6]:
#测试rnn_forward
print("==============测试rnn_forward==============")
np.random.seed(1)
x = np.random.randn(3,10,4)
a0 = np.random.randn(5,10)
Waa = np.random.randn(5,5)
Wax = np.random.randn(5,3)
Wya = np.random.randn(2,5)
ba = np.random.randn(5,1)
by = np.random.randn(2,1)
parameters = {"Waa": Waa, "Wax": Wax, "Wya": Wya, "ba": ba, "by": by}

a, y_pred, caches = rnn_forward(x, a0, parameters)
print("a[4][1] = ", a[4][1])
print("a.shape = ", a.shape)
print("y_pred[1][3] =", y_pred[1][3])
print("y_pred.shape = ", y_pred.shape)
print("caches[1][1][3] =", caches[1][1][3])
print("len(caches) = ", len(caches))


==============测试rnn_forward==============
a[4][1] =  [-0.99999375  0.77911235 -0.99861469 -0.99833267]
a.shape =  (5, 10, 4)
y_pred[1][3] = [0.79560373 0.86224861 0.11118257 0.81515947]
y_pred.shape =  (2, 10, 4)
caches[1][1][3] = [-1.1425182  -0.34934272 -0.20889423  0.58662319]
len(caches) =  2


我们构建了循环神经网络的前向传播函数，这对于某些应用程序来说已经足够好了，但是它还存在梯度消失的问题。当每个输出$y^{\langle t \rangle}$是根据局部的上下文来进行预测的时候，它的效果是比较好的（意思是输入的是$x^{\langle t' \rangle}$，其中$t'$与$t$相隔不是太远）。

接下来我们要构建一个更加复杂的LSTM模型，它可以更好地解决梯度消失的问题，LSTM能够更好地记住一条信息，并且可以在很多时间步中保存。

## 2 长短时记忆（Long Short-Term Memory,LSTM）网络

下面是LSTM的内部结构：

![4](11-4.png)

与上面的RNN例子相类似，我们先来实现一个LSTM单元，只执行一个时间步，然后在循环中调用，以处理所有输入数据。

### 2.1 “门”的介绍
#### 2.1.1 遗忘门
假设我们正在阅读文本中的单词，并希望使用LSTM来跟踪语法结构，比如主语是单数还是复数。如果主语从单数变为复数，我们需要找到一种方法来摆脱我们先前存储的单复数状态的记忆值。在LSTM中，遗忘门是这样做的:

$$
\Gamma_f^{\langle t \rangle} = \sigma(W_f[a^{\langle t-1 \rangle}, x^{\langle t \rangle}] + b_f) \tag{1}
$$

其中，$W_f$是控制遗忘门的权值，我们把$a^{\langle t-1 \rangle}, x^{\langle t \rangle}$连接起来变为$[a^{\langle t-1 \rangle}, x^{\langle t \rangle}]$，然后乘以$W_f$，结果就是得到了一个矢量$\Gamma_f^{\langle t \rangle}$,其值在0与1之间。

这个遗忘门向量将与前一个单元状态$c^{\langle t-1 \rangle}$相乘，因此，如果$\Gamma_f^{\langle t \rangle}$的一个值是0（或者$\approx 0$），则意味着LSTM应该删除对应的信息，如果其中有为1的值，那么LSTM将保留该信息。

#### 2.1.2 更新门
一旦我们“忘记”所讨论的过去的主题是单数，我们需要找到一种方法来更新它，以反映新的主题现在是复数。这里是更新门的公式：

$$
\Gamma_u^{\langle t \rangle} = \sigma(W_u [a^{\langle t-1 \rangle}, x^{\langle t \rangle}] + b_u) \tag{2}
$$

与遗忘门相似，$\Gamma_u^{\langle t \rangle}$向量的值是在0与1之间，为了计算$c^{\langle t \rangle}$，它会与$\tilde c^{\langle t \rangle} $相乘。

#### 2.1.3 更新单元
为了要更新主题，我们需要创建一个新的向量，我们可以将其添加到之前的单元状态中。我们使用的公式是：

$$
\tilde c^{\langle t \rangle} = \tanh(W_c[a^{\langle t-1 \rangle}, x^{\langle t \rangle}] + b_c) \tag{3}
$$

最后，单元的新状态是：

$$
c^{\langle t \rangle} = \Gamma_f^{\langle t \rangle}\ast c^{\langle t - 1 \rangle} + \Gamma_u^{\langle t \rangle} \ast \tilde c^{\langle t \rangle} \tag{4}
$$
#### 2.1.4 输出门
为了决定我们将使用哪种输出，我们将使用以下两个公式：
$$
\Gamma_o^{\langle t \rangle}= \sigma(W_o[a^{\langle t-1 \rangle}, x^{\langle t \rangle}] + b_o)\tag{5}
$$


$$
a^{\langle t \rangle} = \Gamma_o^{\langle t \rangle}* \tanh(c^{\langle t \rangle})\tag{6}
$$
### 2.2 LSTM单元
<span id='3'></span>
我们根据上图来实现一个LSTM单元，步骤如下：

1. 把$a^{\langle t-1 \rangle}, x^{\langle t \rangle}$连接起来变为一个矩阵：$contact = \begin{bmatrix} a^{\langle t -1 \rangle} \\ x^{\langle t \rangle} \\ \end{bmatrix}$
2. 计算公式$a^{\langle t \rangle}$，我们可以使用`sigmoid()`（在rnn_utils内）与`np.tanh()`。

3. 计算预测$y^{\langle t \rangle}$，我们可以使用`softmax()`（在rnn_utils内）。

接下来，你需要**编写代码对函数lstm_cell_forward()进行补全**。

**要点**：
- 依据上述步骤进行实现
- 函数参数及返回值如函数说明所示
- 时刻检查各个变量的形状

In [7]:
def lstm_cell_forward(xt, a_prev, c_prev, parameters):
    """
    根据图4实现一个LSTM单元的前向传播。
    
    参数：
        xt -- 在时间步“t”输入的数据，维度为(n_x, m)
        a_prev -- 上一个时间步“t-1”的隐藏状态，维度为(n_a, m)
        c_prev -- 上一个时间步“t-1”的记忆状态，维度为(n_a, m)
        parameters -- 字典类型的变量，包含了：
                        Wf -- 遗忘门的权值，维度为(n_a, n_a + n_x)
                        bf -- 遗忘门的偏置，维度为(n_a, 1)
                        Wi -- 更新门的权值，维度为(n_a, n_a + n_x)
                        bi -- 更新门的偏置，维度为(n_a, 1)
                        Wc -- 第一个“tanh”的权值，维度为(n_a, n_a + n_x)
                        bc -- 第一个“tanh”的偏置，维度为(n_a, n_a + n_x)
                        Wo -- 输出门的权值，维度为(n_a, n_a + n_x)
                        bo -- 输出门的偏置，维度为(n_a, 1)
                        Wy -- 隐藏状态与输出相关的权值，维度为(n_y, n_a)
                        by -- 隐藏状态与输出相关的偏置，维度为(n_y, 1)
    返回：
        a_next -- 下一个隐藏状态，维度为(n_a, m)
        c_next -- 下一个记忆状态，维度为(n_a, m)
        yt_pred -- 在时间步“t”的预测，维度为(n_y, m)
        cache -- 包含了反向传播所需要的参数，包含了(a_next, c_next, a_prev, c_prev, xt, parameters)
        
    注意：
        ft/it/ot表示遗忘/更新/输出门，cct表示候选值(c tilda)，c表示记忆值。
    """
    
    # 从“parameters”中获取相关值
    Wf = parameters["Wf"]
    bf = parameters["bf"]
    Wi = parameters["Wi"]
    bi = parameters["bi"]
    Wc = parameters["Wc"]
    bc = parameters["bc"]
    Wo = parameters["Wo"]
    bo = parameters["bo"]
    Wy = parameters["Wy"]
    by = parameters["by"]
    
    # 获取 xt 与 Wy 的维度信息
    n_x, m = xt.shape
    n_y, n_a = Wy.shape
    
    ### 在这里填入代码 ###
    
    # 1.连接 a_prev 与 xt
    concat = np.concatenate((a_prev, xt), axis=0)
    
    # 2.根据公式计算ft、it、cct、c_next、ot、a_next
    
    # 遗忘门，公式1
    ft = sigmoid(np.dot(Wf, concat) + bf)
    # 更新门，公式2
    it = sigmoid(np.dot(Wi, concat) + bi)
    # 候选记忆单元，公式3
    cct = np.tanh(np.dot(Wc, concat) + bc)
    # 更新记忆单元，公式4    
    c_next = ft * c_prev + it * cct
    # 输出门，公式5
    ot = sigmoid(np.dot(Wo, concat) + bo)
    # 更新隐藏状态，公式6
    a_next = ot * np.tanh(c_next)
    
    # 3.计算LSTM单元的预测值
    yt_pred = np.dot(Wy, a_next) + by
    yt_pred = np.exp(yt_pred) / np.sum(np.exp(yt_pred), axis=0, keepdims=True)  # 计算softmax
    
    ### 在这里填入代码 ###
    
    # 保存包含了反向传播所需要的参数
    cache = (a_next, c_next, a_prev, c_prev, ft, it, cct, ot, xt, parameters)
    
    return a_next, c_next, yt_pred, cache

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [8]:
#测试lstm_cell_forward
print("==============测试lstm_cell_forward==============")
np.random.seed(1)
xt = np.random.randn(3,10)
a_prev = np.random.randn(5,10)
c_prev = np.random.randn(5,10)
Wf = np.random.randn(5, 5+3)
bf = np.random.randn(5,1)
Wi = np.random.randn(5, 5+3)
bi = np.random.randn(5,1)
Wo = np.random.randn(5, 5+3)
bo = np.random.randn(5,1)
Wc = np.random.randn(5, 5+3)
bc = np.random.randn(5,1)
Wy = np.random.randn(2,5)
by = np.random.randn(2,1)

parameters = {"Wf": Wf, "Wi": Wi, "Wo": Wo, "Wc": Wc, "Wy": Wy, "bf": bf, "bi": bi, "bo": bo, "bc": bc, "by": by}

a_next, c_next, yt, cache = lstm_cell_forward(xt, a_prev, c_prev, parameters)
print("a_next[4] = ", a_next[4])
print("a_next.shape = ", c_next.shape)
print("c_next[2] = ", c_next[2])
print("c_next.shape = ", c_next.shape)
print("yt[1] =", yt[1])
print("yt.shape = ", yt.shape)
print("cache[1][3] =", cache[1][3])
print("len(cache) = ", len(cache))

==============测试lstm_cell_forward==============
a_next[4] =  [-0.66408471  0.0036921   0.02088357  0.22834167 -0.85575339  0.00138482
  0.76566531  0.34631421 -0.00215674  0.43827275]
a_next.shape =  (5, 10)
c_next[2] =  [ 0.63267805  1.00570849  0.35504474  0.20690913 -1.64566718  0.11832942
  0.76449811 -0.0981561  -0.74348425 -0.26810932]
c_next.shape =  (5, 10)
yt[1] = [0.79913913 0.15986619 0.22412122 0.15606108 0.97057211 0.31146381
 0.00943007 0.12666353 0.39380172 0.07828381]
yt.shape =  (2, 10)
cache[1][3] = [-0.16263996  1.03729328  0.72938082 -0.54101719  0.02752074 -0.30821874
  0.07651101 -1.03752894  1.41219977 -0.37647422]
len(cache) =  10


### 2.3 LSTM的前向传播
<span id='4'></span>
我们已经实现了LSTM单元的一个时间步的前向传播，现在我们要对LSTM网络进行前向传播进行计算
![](11-5.png)

我们来实现lstm_forward()，然后运行$T_x$个时间步。

注意：$c^{\langle 0 \rangle}$使用0来初始化。

接下来，你需要**编写代码对函数lstm_forward()进行补全**。

**要点**：
- 依据上述步骤进行实现
- 函数参数及返回值如函数说明所示
- 时刻检查各个变量的形状

In [9]:
def lstm_forward(x, a0, parameters):
    """
    根据图5来实现LSTM单元组成的循环神经网络
    
    参数：
        x -- 所有时间步的输入数据，维度为(n_x, m, T_x)
        a0 -- 初始化隐藏状态，维度为(n_a, m)
        parameters -- python字典，包含了以下参数：
                        Wf -- 遗忘门的权值，维度为(n_a, n_a + n_x)
                        bf -- 遗忘门的偏置，维度为(n_a, 1)
                        Wi -- 更新门的权值，维度为(n_a, n_a + n_x)
                        bi -- 更新门的偏置，维度为(n_a, 1)
                        Wc -- 第一个“tanh”的权值，维度为(n_a, n_a + n_x)
                        bc -- 第一个“tanh”的偏置，维度为(n_a, n_a + n_x)
                        Wo -- 输出门的权值，维度为(n_a, n_a + n_x)
                        bo -- 输出门的偏置，维度为(n_a, 1)
                        Wy -- 隐藏状态与输出相关的权值，维度为(n_y, n_a)
                        by -- 隐藏状态与输出相关的偏置，维度为(n_y, 1)
        
    返回：
        a -- 所有时间步的隐藏状态，维度为(n_a, m, T_x)
        y -- 所有时间步的预测值，维度为(n_y, m, T_x)
        caches -- 为反向传播的保存的元组，维度为（【列表类型】cache, x)）
    """
    
    # 初始化“caches”
    caches = []
    
    ### 在这里填入代码 ###
    
    # 获取 x 与 Wy 的维度信息
    n_x, m, T_x = x.shape
    n_y, n_a = parameters["Wy"].shape
    
    # 使用0来初始化“a”、“c”、“y”
    a = np.zeros((n_a, m, T_x))
    c = np.zeros((n_a, m, T_x))
    y = np.zeros((n_y, m, T_x))
    
    # 初始化“a_next”、“c_next”
    a_next = a0
    c_next = np.zeros((n_a, m))
    
    # 遍历所有的时间步
    for t in range(T_x):
        # 更新下一个隐藏状态，下一个记忆状态，计算预测值，获取cache
        a_next, c_next, yt_pred, cache = lstm_cell_forward(x[:, :, t], a_next, c_next, parameters)
        
        # 保存新的下一个隐藏状态到变量a中
        a[:, :, t] = a_next
        
        # 保存预测值到变量y中
        y[:, :, t] = yt_pred
        
        # 保存下一个单元状态到变量c中
        c[:, :, t] = c_next
        
        # 把cache添加到caches中
        caches.append(cache)
    
    # 保存反向传播需要的参数
    caches = (caches, x)
    
    ### 在这里填入代码 ###
    
    return a, y, c, caches

In [10]:
#测试lstm_forward
print("==============测试lstm_forward==============")
np.random.seed(1)
x = np.random.randn(3,10,7)
a0 = np.random.randn(5,10)
Wf = np.random.randn(5, 5+3)
bf = np.random.randn(5,1)
Wi = np.random.randn(5, 5+3)
bi = np.random.randn(5,1)
Wo = np.random.randn(5, 5+3)
bo = np.random.randn(5,1)
Wc = np.random.randn(5, 5+3)
bc = np.random.randn(5,1)
Wy = np.random.randn(2,5)
by = np.random.randn(2,1)

parameters = {"Wf": Wf, "Wi": Wi, "Wo": Wo, "Wc": Wc, "Wy": Wy, "bf": bf, "bi": bi, "bo": bo, "bc": bc, "by": by}

a, y, c, caches = lstm_forward(x, a0, parameters)
print("a[4][3][6] = ", a[4][3][6])
print("a.shape = ", a.shape)
print("y[1][4][3] =", y[1][4][3])
print("y.shape = ", y.shape)
print("caches[1][1[1]] =", caches[1][1][1])
print("c[1][2][1]", c[1][2][1])
print("len(caches) = ", len(caches))

==============测试lstm_forward==============
a[4][3][6] =  0.17211776753291663
a.shape =  (5, 10, 7)
y[1][4][3] = 0.9508734618501101
y.shape =  (2, 10, 7)
caches[1][1[1]] = [ 0.82797464  0.23009474  0.76201118 -0.22232814 -0.20075807  0.18656139
  0.41005165]
c[1][2][1] -0.8555449167181982
len(caches) =  2


在现代深度学习框架中，我们只需要实现前向传播，框架负责反向传播。因此这本节练习中，对于循环神经网络的反向传播我们不多做探讨。